## YoloV8 (detection on P0) 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import tqdm
import mmcv
import torch
import json
import numpy as np
import pandas as pd
import multiprocessing

from ultralytics import YOLO

from services.ultralytics_utils import to_detections

## Constants

In [ ]:
DATASET_PATH = '/home/ubuntu/git/B2B_ALGO/datasets/ab_car_validation_germany'
GT_PATH = f'{DATASET_PATH}/annotations/gt.tsv'

RESULTS_PATH = '/home/ubuntu/git/B2B_ALGO/results/ALGO1'

## Data

In [ ]:
gt_df = pd.read_csv(GT_PATH, sep='\t')
gt_df

In [ ]:
paths  = f'{DATASET_PATH}/images' + '/' + np.unique(gt_df.name.values)
paths[:1], len(paths)

## Algo

In [ ]:
%%time
model = YOLO('weights/yolov8x.pt')

In [ ]:
predict_fn = lambda path: model.predict(path, imgsz=(1920, 3840), conf = 0.1, iou = 0.7, classes=[0, 1, 2, 3, 5,7], line_width=1, show=False, save=False, verbose=False)[0]

records = map(lambda path: (path, predict_fn(path)), paths)

In [ ]:
%%time
det_df = to_detections(records)
det_df

## Save Detections

In [ ]:
!mkdir -p {RESULTS_PATH}

det_df.to_csv(f'{RESULTS_PATH}/det.tsv', sep='\t',index=False)